In [5]:
from tkinter import *
from PIL import ImageTk, Image
import threading
import cv2 


# Run this Cell twice, I'm not exactly sure why it works on the second time, but it will overlay the squares this way.

In [11]:
import tkinter as tk

class Controller:
    def __init__(self, root, img_size=(300, 550)):
        # Window stuff
        self.root = root
        self.root.title("Controller")
        root.attributes('-fullscreen', True)
        self.window_width = int(window.winfo_screenwidth() / 58)
        self.window_height = (self.window_width * 2)
        self.img_size = img_size
        # threading stuff
        self.t1 = threading.Thread(target=self.flash_top)
        self.feed = threading.Thread(target=self.open_camera)
        self.t3 = threading.Thread(target=self.flash_bottom)
        
        
        self.screen_width = self.root.winfo_screenwidth()
        self.screen_height = self.root.winfo_screenheight()
        self.cap = cv2.VideoCapture(0)
        self.cap.set(cv2.CAP_PROP_FRAME_WIDTH, self.window_width)
        self.cap.set(cv2.CAP_PROP_FRAME_HEIGHT, self.window_height)
        
          
        # Set the width and height 
        
        self.canvas = tk.Canvas(self.root, width=self.screen_width, height=self.screen_height)
        self.canvas.place(anchor='nw', x=0, y=0)
        self.root.bind('<Return>', lambda e: self.root.quit()) 
        
        self.rect_width = 130
        self.rect_height = 130
        self.up=None
        self.down=None
        self.is_on_top = False
        self.is_on_bottom = False
        self.image_id = None 
        
        self.threading()

    def run(self):
        self.root.mainloop()

    def threading(self):
        self.t1.start()
        self.t3.start()
        self.feed.start()
        
    def open_camera(self):
        # Capture the video frame by frame
        _, frame = self.cap.read()
        frame = cv2.resize(frame, (self.screen_width, self.screen_height), interpolation=cv2.INTER_LINEAR)
        # Convert image from one color space to other
        opencv_image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGBA)
        
    
        # Capture the latest frame and transform to image
        self.captured_image = Image.fromarray(opencv_image)
    
        # Convert captured image to photoimage
        self.photo_image = ImageTk.PhotoImage(image=self.captured_image)
    
        # Displaying photoimage in the label
        self.canvas.photo_image = self.photo_image
    
        # Configure image in the label
        if self.image_id:       
            # replace image in PhotoImage on canvas
            self.canvas.itemconfig(self.image_id, image=self.photo_image)
            
        else:
            # create first image on canvas and keep its ID
            self.image_id = self.canvas.create_image((0,0), image=self.photo_image, anchor='nw')
            # resize canvas
            self.canvas.configure(width=self.screen_width, height=self.screen_height)
            
    
        #self.update_flash_rectangles()
        # Repeat the same process after every 10 seconds
        self.root.after(10, self.open_camera)

    def flash_top(self):
        self.is_on_top = not self.is_on_top
        self.update_flash_rectangles()
        # 12 hertz
        self.root.after(83, self.flash_top)

    def flash_bottom(self):
        self.is_on_bottom = not self.is_on_bottom
        self.update_flash_rectangles()
        # 15 hertz
        self.root.after(66, self.flash_bottom)
        
    def update_flash_rectangles(self):       
        # Update the rectangle colors based on the flashing state
        center_x = self.screen_width // 2
        center_y = self.screen_height // 2

        # Create or update the top rectangle (flashes on and off)
        if not self.up:
            self.up = self.canvas.create_rectangle(
                center_x - self.rect_width // 2,
                center_y - 400 - self.rect_height // 2,
                center_x + self.rect_width // 2,
                center_y - 400 + self.rect_height // 2,
                fill='black'
            )
        self.canvas.itemconfig(self.up, fill='white' if self.is_on_top else 'black')

        # Create or update the bottom rectangle (flashes on and off)
        if not self.down:
            self.down = self.canvas.create_rectangle(
                center_x - self.rect_width // 2,
                center_y + 400 - self.rect_height // 2,
                center_x + self.rect_width // 2,
                center_y + 400 + self.rect_height // 2,
                fill='black'
            )
        self.canvas.itemconfig(self.down, fill='white' if self.is_on_bottom else 'black')





# Initializes the Code

window = tk.Tk()
RC_Controller = Controller(window, img_size=(300, 525))
RC_Controller.run()
